In [ ]:
from itertools import chain
from pathlib import Path

from pydantic import BaseModel

In [ ]:
albums = Path("D:/Google takeout/2024-11-20-photos")
people = list((albums / "people").iterdir())
undated = list((albums / "undated").iterdir())
trash = albums / "trash"
print(
    f"{len(people)=}",
    f"{len(undated)=}",
    f"Trash: {trash.relative_to(albums).as_posix()}",
    sep="\n",
)

In [ ]:
class Album(BaseModel):
    path: Path

    @property
    def metadata(self) -> Path:
        return self.path / "metadata.json"

    @property
    def file_metadata(self) -> list[Path]:
        return list(self.path.glob("[!metadata]*.json"))

    @property
    def files(self) -> list[Path]:
        return list(self.path.glob("[!metadata]*[!.json]"))

    @property
    def missing(self) -> list[Path]:
        return [
            path.with_name(path.stem)
            for path in self.file_metadata
            if not path.with_name(path.stem).exists()
        ]


dated = {
    album.name: Album(path=album)
    for album in [*(albums / "dated").iterdir(), *(albums / "dated-done").iterdir()]
}
all_years = albums / "years"
years = {album.name: Album(path=album) for album in all_years.iterdir()}
photos = list(chain.from_iterable(album.files for album in years.values()))
print(
    f"{len(dated)=}",
    f"{len(years)=}",
    f"{len(photos)=}",
    sep="\n",
)

In [ ]:
dated_photo_names = [
    path.name for path in chain.from_iterable(album.files for album in dated.values())
]
[path.name for path in trash.iterdir() if path.name in dated_photo_names]

In [ ]:
album = dated["2008-10-25 Pool"]
missing = [
    path.with_name(path.stem)
    for path in album.file_metadata
    if not path.with_name(path.stem).exists()
]
missing_names = [missing_photo.name for missing_photo in missing]
[
    photo.relative_to(all_years).as_posix()
    for photo in photos
    if photo.name in missing_names
]

In [ ]:
found: dict[str, list[str]] = {}
for name, album in dated.items():
    missing = [
        path.with_name(path.stem)
        for path in album.file_metadata
        if not path.with_name(path.stem).exists()
    ]
    missing_names = [missing_photo.name for missing_photo in missing]
    found[name] = [
        photo.relative_to(all_years).as_posix()
        for photo in photos
        if photo.name in missing_names
    ]
found